In [1251]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import struct

In [1252]:

# generate a,b
def generate_vectors(n):
    a_uniform_distribution = np.random.uniform(low=-1_0000, high=1_0000, size=(1, n)) 
    b_uniform_distribution = np.random.choice([0, 1], size=(n, 1))

    return a_uniform_distribution, b_uniform_distribution

In [1253]:
#base
def multiply_vectors(a,b):
    a_bfloat16 = a.astype(np.float16)
    result = np.dot(a_bfloat16, b)
    return result

In [1254]:
#向量中元素转化为二进制操作
def Modify_tensor(tensor):
    bf_list = tensor.detach().numpy().tolist() # 转成numpy变量再转换成list    
    binary_array = []

    for bf_nums in bf_list:
        for bf_num in bf_nums:
            float_bytes = struct.pack('>f', bf_num)
            byte_array = ''.join(f'{byte:08b}' for byte in float_bytes)
            binary_array.append(byte_array)
    return binary_array

#将8位补码二进制字符串转换为整数
def twos_complement_to_int(binary_str):
    if binary_str == "":
        return 0

    if binary_str[0] == '1':
        inverted = ''.join('1' if bit == '0' else '0' for bit in binary_str)  
        int_value = int(inverted, 2) + 1  
        return -int_value  
    else:
        return int(binary_str, 2)  

In [1255]:
#ReDCIM
def ReDCIM_multiply(a,b):
    a_tensor = torch.from_numpy(a)
    b_tensor = torch.from_numpy(b)
    a_fp32 = a_tensor.float()
    a_modified = Modify_tensor(a_fp32)

    ##预对齐
    a_exp = []
    for number in a_modified:
        extracted_part = number[1:9]
        a_exp.append(extracted_part)
    a_exp_int = [int(binary, 2) for binary in a_exp]
    a_exp_max = max(a_exp_int)
    a_difference = [a_exp_max - value for value in a_exp_int]
    print(a_exp)

    a_mantissa = []
    for number in a_modified:
        if number[0] == '1':
            inverted_part ='0' + ''.join('1' if bit == '0' else '0' for bit in number[9:15]) #取反码
            complement = format(int(inverted_part, 2) + 1, '07b') #取补码
            extracted_part = number[0] + complement #加符号位
        else:
            extracted_part = number[0] + '1' + number[9:14]
        a_mantissa.append(extracted_part)
    print(a_mantissa)

    a_shifted_mantissa_values = []
    for mant, diff in zip(a_mantissa, a_difference):
        sign = int(mant[0], 2) #将mantissa 的首位取为符号位，并转化为int类型
        if diff > 0:
            shifted_value_binary = (sign * '1' if sign == 1 else '0') * diff + mant[:-diff] #str类型拼接
        else:
            shifted_value_binary = mant
        a_shifted_mantissa_values.append(shifted_value_binary)
    print(a_shifted_mantissa_values)

    product_mantissa = [a * b for a, b in zip(a_shifted_mantissa_values, b_tensor)]
    product_mantissa_int= [twos_complement_to_int(product) for product in product_mantissa]
    sum_product_mantissa = sum(product_mantissa_int)
    sign = 1 if sum_product_mantissa < 0 else 0
    
    combined_result = ((-1) ** sign) * (2 ** (a_exp_max - 127)) * sum_product_mantissa
    return combined_result

In [1256]:
#vectors
n = 4
a,b = generate_vectors(n)
print(a)
print(b)

result_standard = multiply_vectors(a, b)
print("Result standard:\n", result_standard)

result_ReDCIM = ReDCIM_multiply(a, b)
print("Result ReDCIM:\n", result_ReDCIM)

[[ -421.70414441 -1763.61997011 -5167.28734286 -5880.07533226]]
[[0]
 [0]
 [0]
 [0]]
Result standard:
 [[0.]]
['10000111', '10001001', '10001011', '10001011']
['10010111', '10010010', '10110000', '10100101']
['11111001', '11100100', '10110000', '10100101']
Result ReDCIM:
 0
